In [ ]:
import sys
sys.path.append('/tf/data')

import os
import torch
import shutil

from general_func import load_dataset
from Classification.class_functions import split_ds, concat_data, plot_loss_acc
from Classification.Custom_GridSearch import grid_search

In [ ]:
base_params = {
    'run': 'Search_2',
    'model_save_path': '/tf/data/Classification/ConvNeXt/Grid_Search3/',
    'model_save_freq_epochs': 1,
    'seed' : 42,
    
    'num_epochs': 150,
    'warmup_epochs': 10,
    'early_stop': 15,
    
    'loader_workers': 2,
    'apply_class_weights': True,
    
    'num_ops': 8,
    'magnitude': 20,
    'offset': True,
    'ratio': 0.5,
    'mode': 1,
    'prob': 0.5
}


model1 = {**base_params,
          'learning_rate': 0.004,
          'weight_decay': 0.005,
          'batch_size': 8,
          'drop_rate': 0.5}

model2 = {**base_params,
          'learning_rate': 0.004,
          'weight_decay': 0.01,
          'batch_size': 8,
          'drop_rate': 0.5}

model3 = {**base_params,
          'learning_rate': 0.004,
          'weight_decay': 0.005,
          'batch_size': 16,
          'drop_rate': 0.5}

print('Device:', torch.device("cuda" if torch.cuda.is_available() else "cpu"))

In [ ]:
#Load data and create dataloaders
ds_pos = load_dataset(positive=True)
train_scans_pos, val_scans_pos, test_scans_pos = split_ds(ds_pos, train_split = 0.70, val_split = 0.30, seed = base_params['seed'])
ds_neg = load_dataset(positive=False)
train_scans_neg, val_scans_neg, test_scans_neg = split_ds(ds_neg, train_split = 0.70, val_split = 0.30, seed = base_params['seed'])

search_n = 0
for params in [model1, model2, model3]:
    train_loader = concat_data(train_scans_pos, train_scans_neg, batch_size=params['batch_size'], workers=params['loader_workers'])
    val_loader = concat_data(val_scans_pos, val_scans_neg, batch_size=params['batch_size'], workers=params['loader_workers'])
    test_loader = concat_data(test_scans_pos, test_scans_neg, batch_size=params['batch_size'], workers=params['loader_workers'])
    for num_ops in [4, 8, 12]:
        for magnitude in [10, 20, 30]:
            for offset in [True, False]:
                for ratio in [.5, .7]:
                    for probability in [.5, .7]:
                        search_n += 1
                        if search_n < 111:
                            continue
                        print('Training model:', search_n) 
                        params['num_ops'] = num_ops
                        params['magnitude'] = magnitude
                        params['offset'] = offset
                        params['ratio'] = ratio
                        params['prob'] = probability
    
                        _, train_losses, val_losses, val_accuracy = grid_search(params, search_n, train_loader, val_loader, test_loader, ds_pos, ds_neg, log_path = params['model_save_path']+params['run'])
                        
                        #Delete irellevant searches:
                        folder = params['model_save_path']+params['run']+'/'+str(search_n)
                        if min(val_losses) > 0.25:
                            if os.path.exists(folder):
                                shutil.rmtree(folder)
                                print("Folder and its contents deleted successfully.")
                        else:
                            plot_loss_acc(train_losses, val_losses, val_accuracy, save_dir = folder)